### Start lokign at eye-tracking data from risk task

1st: clean up data, put into right format!

This is a library that Tomas and Jan-Willem and I built to analyze data from the Eyelink.
https://github.com/Gilles86/hedfpy

These are some analyses on Miguel's data using that stuff.
https://github.com/Gilles86/riskeye

Careful, you need the edf2asc-binary (computer program). I think for that you need officially need a Eyelink account.

And look for the 'EyeLink Developer’s Kit'

But you can also find a binary in the root directory of this repo
https://github.com/Gilles86/riskeye

I think it's for Linux/Ubuntu, so you can run it on Sciencecloud or Docker.

### file conversion

on sciencecloud `~/git/riskeye/edf2asc`
created `/mnt_03/ds-dnumrisk/09_14_11.asc`

## renaming to organize

In [124]:
# renaming & reorganizing files

import os
import pandas as pd
import os.path as op
from datetime import datetime

bids_folder = '/Users/mrenke/data/ds-dnumrisk'
pati_eye = op.join(bids_folder, 'sourcedata/eyetracking_risk')
pati_eye_new = op.join(bids_folder, 'sourcedata/eyetracking_risk_renames')
#os.makedirs(pati_eye_new, exist_ok=True)

fns = os.listdir(pati_eye)

In [125]:
scan_table = op.join(bids_folder, 'add_tables/subjects_recruit&scan_scanned_final.csv')
df = pd.read_csv(scan_table, header=1, usecols=[0,1,2,3,4],nrows=66)#.dropna() #,#sheet_name='scanned', header=1)  
df['date'] = pd.to_datetime(df['date'],dayfirst=True)
df['subject ID'] = df['subject ID'].astype(int)
#df.set_index('subject ID', inplace=True)
#df


In [116]:
file_path = os.path.join(pati_eye,'13_58_41.edf' ) #fns[3]
mod_time = os.path.getmtime(file_path)

mod_time = datetime.fromtimestamp(mod_time)## _%H-%M-%S
print(file_path + ' created ' + mod_time.strftime('%d-%m-%Y') )

/Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_58_41.edf created 06-02-2023


In [117]:
file_year = mod_time.year
file_month = mod_time.month
file_day = mod_time.day

matching_row = df[df['date'].apply(lambda x: (x.year == file_year) and (x.month == file_month) and (x.day == file_day))]


In [126]:
fns_2days = []
fns_unmatched = []

for fn in fns:
    full_fn = os.path.join(pati_eye,fn) 
    mod_time = os.path.getmtime(full_fn)
    mod_time = datetime.fromtimestamp(mod_time)
    matching_row = df[df['date'].apply(lambda x: (x.year == mod_time.year) and (x.month == mod_time.month) and (x.day == mod_time.day))]

    if len(matching_row['subject ID'].values) == 1: # only one match!
        sub_id = matching_row['subject ID'].values[0]
        new_filename = f"sub-{sub_id}_{mod_time.day}-{mod_time.month}-{mod_time.year}.edf"
        full_fn_new = os.path.join(pati_eye_new,new_filename)
        os.rename(full_fn, full_fn_new) # Renames a file and can also move it to a new location if the new name includes a different path.
        print(f"Renamed {full_fn} to {new_filename}")
        
    elif len(matching_row['subject ID'].values) == 0:
        fns_unmatched.append(fn)
    elif len(matching_row['subject ID'].values) == 2:
        fns_2days.append(fn)



Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_05_31.edf to sub-51_16-1-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/10_54_48.edf to sub-12_12-4-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_58_41.edf to sub-1_6-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/16_15_12.edf to sub-1_6-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/16_23_07.edf to sub-11_15-3-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/17_12_44.edf to sub-52_16-2-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/14_29_27.edf to sub-24_13-6-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/10_43_53.edf to sub-59_8-4-2024.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/09_38_25.edf to sub-6_28-2-2023.edf
Renamed /Users/mrenke/data/ds-dnumrisk/sourcedata/eyetracking_risk/13_

In [ ]:
## format ?
# first non-symbolic, then symbolic

In [127]:
fns_2days

['eye002.log',
 '11_32_33.edf',
 '10_49_31.edf',
 '12_03_33.edf',
 '16_20_56.edf',
 'eye003.log',
 '16_12_21.edf',
 'eye007.log',
 '11_16_37.edf',
 '11_02_56.edf',
 '15_46_18.edf',
 '16_10_56.edf',
 '11_15_06.edf',
 '11_42_17.edf',
 '16_03_11.edf',
 '10_10_56.edf',
 '11_44_48.edf',
 '11_02_08.edf',
 '11_11_48.edf',
 '15_31_41.edf',
 '16_03_10.edf',
 '11_21_19.edf',
 '11_00_14.edf',
 '12_12_08.edf',
 '16_12_58.edf',
 '16_54_30.edf',
 '11_33_18.edf',
 '10_01_34.edf',
 '15_53_33.edf',
 '10_31_40.edf',
 '11_12_17.edf',
 '11_23_22.edf',
 '10_49_59.edf',
 '16_15_57.edf',
 '10_18_39.edf',
 '10_52_52.edf',
 '16_28_49.edf',
 '10_40_08.edf',
 '10_40_35.edf',
 '16_06_58.edf',
 '17_03_23.edf',
 'eye009.log',
 '11_11_34.edf',
 '16_37_41.edf',
 '11_31_44.edf',
 '11_04_02.edf',
 '11_21_43.edf',
 '16_02_06.edf',
 'trackerui.log']

In [128]:
import re
from io import StringIO
import argparse
import os.path as op
import os
import subprocess
import gzip

import glob
import pandas as pd
import shutil

In [ ]:


def extract_data(subject, root_folder='/data'):
    subject = int(subject)

    dir = get_subject_folder(subject, root_folder) 

    for run in runs:
        fn = op.join(dir, f'Rs{subject:02d}rn{run:02d}.edf')
        asc_fn = fn.replace('.edf', '.asc')
        gaze_target_fn = fn.replace('.edf', '.gaz.gz')

        # get gaze
        cmd = f'edf2asc  -t -y -z -v -s -vel {fn}'

        subprocess.run(cmd, shell=True)

        with open(asc_fn, 'rb') as asc_file, gzip.open(gaze_target_fn, 'wb') as target_file:
                target_file.writelines(asc_file)

        os.remove(asc_fn)

        # get messages
        cmd = f'edf2asc  -t -y -z -v -e {fn}'
        msg_target_fn = fn.replace('.edf', '.msg.gz')

        subprocess.run(cmd, shell=True)

        with open(asc_fn, 'rb') as asc_file, gzip.open(msg_target_fn, 'wb') as target_file:
                target_file.writelines(asc_file)

        os.remove(asc_fn)